<a href="https://colab.research.google.com/github/manish2021iitd/Deep-Learning/blob/main/DLassignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [2]:
import wandb

In [3]:
wandb.login(key='e3c892d4f8c9cd9b9043d31938ad090f0a32cec1')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import torch
import torch.nn as nn

#class of convolution neural network
class CNN(nn.Module):
    #intitialer for parameters
    def __init__(self, num_conv_layers, num_classes):
        super(CNN, self).__init__()
        #convolutional layers
        self.conv_layers = nn.ModuleList()
        for _ in range(num_conv_layers):
            self.conv_layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, padding=1))
            self.conv_layers.append(nn.ReLU())
            self.conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

        #dense layer
        self.dense = nn.Linear(out_size_after_pooling * out_size_after_pooling * out_channels, num_classes)

    #forward pass
    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        x = self.dense(x)
        return x
